# Python для сбора и обработки данных

Автор: Валентин Бирюков

## Задача - продолжаем собирать данные =)

Теперь уже не с крупных сайтов компаний, а с "площадок". Рассмотрим это на примере VK.

Пусть мы хотим как то изучить целивую аудиторию/следить за активностью в социальных сетях вокруг бренда, тд.

У части таких платформ есть специальные "ручки" - инструменты, с помощью которых можно направлять запросы к порталу, не используя при этом танцы вокруг выцепления информации их хтмл кода страницы. Так и поступим.


Документация: https://vk.com/dev

# Задача 0 - аааа как с этим работать

In [3]:
! pip install vk

In [31]:
import vk
API_TOKEN_VK = 'TOKEN'
API_TOKEN_VK_2 = 'TOKEN'
session = vk.Session(access_token=API_TOKEN_VK)
my_api = vk.API(session, v='5.21')

In [32]:
session_2 = vk.Session(access_token=API_TOKEN_VK_2)
my_api_2 = vk.API(session, v='5.21')

In [9]:
86400/60/60/24

1.0

# Задача 1 - Собрать три круга друзей

In [10]:
user_to_prepare = 'valentin.biryukov' 

user_info = my_api.users.get(user_ids=user_to_prepare)
print(user_info)
start_id = user_info[0]['id']

[{'id': 21785564, 'first_name': 'Валентин', 'last_name': 'Бирюков'}]


In [11]:
friends_first_hand = my_api.friends.get(user_id=start_id)
print(friends_first_hand)
friends_first_hand_id = friends_first_hand['items']

{'count': 86, 'items': [398045, 409037, 1538693, 2956433, 4228387, 4516987, 5317749, 5334461, 5412679, 5417191, 10348887, 10873145, 11164850, 13275626, 14204056, 15945852, 17336364, 20580174, 22304277, 24874560, 24976336, 30606127, 30816900, 34248421, 34746243, 36623405, 38459100, 38830624, 38900678, 44898863, 47402039, 47672038, 49915195, 50489710, 51121895, 52203550, 53847565, 54782818, 56726638, 59536344, 61657552, 62902082, 66865352, 68603392, 68812424, 71542865, 72937909, 76702064, 85427885, 88293148, 95437665, 97341087, 106480301, 107098336, 107916896, 107920824, 112162069, 114355530, 114776804, 114904924, 122307165, 124821194, 125824662, 145762347, 148396546, 152426005, 156561760, 161176665, 165968752, 177372466, 209463582, 209662408, 213734038, 227227659, 262217495, 271656949, 275633587, 283975933, 295142987, 317187625, 321409001, 370549931, 560118043, 562252903, 566953060, 574103916]}


In [21]:
from time import sleep

In [35]:
apis = [my_api, my_api_2]

In [39]:
friends_second_hand = []
for i, fr in enumerate(friends_first_hand_id):
    api = apis[i%len(apis)]
    friends_second_hand.extend(api.friends.get(user_id=fr)['items'])
    sleep(0.3)
friends_second_hand = list(set(friends_second_hand))

In [42]:
friends_3_hand = []
for i, fr in enumerate(friends_second_hand[:300]):
    api = apis[i%len(apis)]
    try:
        friends_3_hand.extend(api.friends.get(user_id=fr)['items'])
        sleep(0.3)
    except:
        pass
friends_3_hand = list(set(friends_3_hand))

# Задача 2 - посчитаем "охват" 

In [48]:
group = -72495085
post_ids = [item['id'] for item in my_api.wall.get(owner_id=group, count=100)['items']]
len(post_ids)

100

In [49]:
comments = []
for post_id in post_ids:
    info = my_api.wall.getComments(owner_id=group, post_id=post_id)['items']
    topic = [
        {
            'user_id': item['from_id'],
            'comment_id': item['id']
        } for item in info
    ]
    comments.extend(topic)
    sleep(0.3)
len(comments)

990

In [55]:
likes_users_post = []
for post_id in post_ids:
    likes_users_post.extend([my_api.likes.getList(type='post', owner_id=group, item_id=post_id)['items']])
    sleep(0.3)


In [ ]:
likes_users_comment = []
for comments_id in comments:
    likes_users_comment.extend([my_api.likes.getList(type='comment', owner_id=group, item_id=comments_id['comment_id'])['items']])
    sleep(0.3)

In [ ]:
commentators = [item['comment_id'] for item in comments]

In [ ]:
print(likes_users_post)
print(len(set(likes_users_post)))

print('-------------')

print(likes_users_comment)
print(len(set(likes_users_comment)))

print('-------------')

print(likes_users_post+likes_users_comment+commentators)
print(len(set(likes_users_post+likes_users_comment+commentators)))

# Задача 3 - "создадим портрет ЦА"